### Описание проекта

Цель настоящего исследования - изучение результатов опроса, проведенного с целью определить уровень потребительской лояльности телекоммуникационной компании среди клиентов из России. 

Результатом изучения должны стать ответы на следующие вопросы:
- Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

В ходе исследования данные будут собраны из базы с помощью SQL-запроса, для их визуализации будет подготовлен дашборд. Ответы на вопросы будут представлены в виде презентации.

### SQL-запрос для выгрузки информации из базы данных

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT
user_id,
lt_day,
IIF(lt_day <= 365, 'Новый', 'Старый') AS is_new,
age,
CASE WHEN gender_segment = 1
        THEN 'Женский'
    WHEN gender_segment = 0
        THEN 'Мужской'
    ELSE 'n/a'
END AS 'gender_segment',
os_name,
cpe_type_name,
country,
city,
SUBSTR(ags.title, 4) AS age_segment,
SUBSTR(ts.title, 4) AS traffic_segment,
SUBSTR(ls.title, 4) AS lifetime_segment,
nps_score,
CASE WHEN nps_score BETWEEN 9 AND 10
        THEN 'Сторонники'
    WHEN nps_score  BETWEEN 7 AND 8
        THEN 'Нейтралы'
    ELSE 'Противники'
END AS nps_group
FROM user INNER JOIN location AS loc ON user.location_id = loc.location_id
INNER JOIN age_segment AS ags ON user.age_gr_id = ags.age_gr_id
INNER JOIN traffic_segment AS ts ON user.tr_gr_id = ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON user.lt_gr_id = ls.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женский,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонники
1,A001WF,2344,Старый,53.0,Мужской,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонники
2,A003Q7,467,Старый,57.0,Мужской,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонники


In [5]:
len(df.loc[df['gender_segment'] == 'n/a'])

1301

In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)